# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [11]:
# import libraries
import numpy as np
import pandas as pd
import sys
from sqlalchemy import create_engine
import nltk

nltk.download('punkt')
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

import re

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier

from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

from sklearn.neighbors import KNeighborsClassifier

[nltk_data] Downloading package punkt to /home/salman1567/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/salman1567/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
# load data from database
engine = create_engine('sqlite:///disaster.db')
df = pd.read_sql_table('disaster', engine)
X = df.iloc[:,1].dropna()
Y = df.iloc[:,4:]

print(Y.head())

   related  request  offer  aid_related  medical_help  medical_products  \
0        1        0      0            0             0                 0   
1        1        0      0            1             0                 0   
2        1        0      0            0             0                 0   
3        1        1      0            1             0                 1   
4        1        0      0            0             0                 0   

   search_and_rescue  security  military  child_alone  ...  aid_centers  \
0                  0         0         0            0  ...            0   
1                  0         0         0            0  ...            0   
2                  0         0         0            0  ...            0   
3                  0         0         0            0  ...            0   
4                  0         0         0            0  ...            0   

   other_infrastructure  weather_related  floods  storm  fire  earthquake  \
0                    

### 2. Write a tokenization function to process your text data

In [13]:
def tokenize(text):
    
    clean_tokens = []
    
    # lowercasing and removing punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower().strip())
    tokens = word_tokenize(text)
    # Remove stop words + stem
    stemmer = PorterStemmer()
    
    
    stop_words = stopwords.words("english")
    clean_tokens = [ stemmer.stem(word) for word in tokens if word not in stop_words]

    return clean_tokens


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [14]:
# Try other estimators to see if performance is better
pipeline = Pipeline([
    #('tokenize', tokenize())
    ('count-vector', CountVectorizer(tokenizer=tokenize)),
    ('tf-idf', TfidfTransformer()),
    #('tf-idf', TfidfVectorizer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_jobs=-1)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=1)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('count-vector',
                 CountVectorizer(tokenizer=<function tokenize at 0x7f240c513640>)),
                ('tf-idf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(n_jobs=-1)))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [16]:
col_names = Y.columns.unique()
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred, target_names=col_names))



                        precision    recall  f1-score   support

               related       0.77      0.97      0.86      6545
               request       0.45      0.04      0.08      1520
                 offer       0.00      0.00      0.00        34
           aid_related       0.45      0.22      0.29      3582
          medical_help       0.12      0.01      0.01       672
      medical_products       0.00      0.00      0.00       441
     search_and_rescue       0.25      0.00      0.01       230
              security       0.00      0.00      0.00       155
              military       1.00      0.00      0.01       308
           child_alone       0.00      0.00      0.00         0
                 water       0.12      0.01      0.01       568
                  food       0.25      0.01      0.02       973
               shelter       0.16      0.01      0.01       775
              clothing       0.00      0.00      0.00       134
                 money       0.00      

/home/salman1567/code/Data Science - Machine Learning Projects/Data Science Portfolio/Diasaster Response Classification/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/salman1567/code/Data Science - Machine Learning Projects/Data Science Portfolio/Diasaster Response Classification/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/salman1567/code/Data Science - Machine Learning Projects/Data Science Portfolio/Diasaster Response Classification/env/lib/python3.10/site-packages/sklea

### 6. Improve your model
Use grid search to find better parameters. 

In [17]:
parameters = 

cv = 

SyntaxError: invalid syntax (3277122076.py, line 1)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.